In [10]:
import numpy as np
from scipy.optimize import minimize
import math 
# Define constants
L = 1852 * 4
gamma = np.sin(np.radians(31.5)) / (np.sin(np.radians(31.5)) + np.sin(np.radians(28.5)))
alpha = np.radians(1.5)
theta = np.radians(120)

def objective(z):
    return math.sqrt(len(z)**2)

def constraint1(z):
    y = np.cumsum(z)
    D = (2 * 1852 * np.tan(np.radians(1.5)) + 110) - y * np.tan(np.radians(1.5))
    W = (np.sin(theta) * (np.cos(alpha) ** 2)) / (np.cos(theta / 2 + alpha) * np.cos(theta / 2 - alpha)) * D
    y_next=np.append(y[1:],0)
    W_next=np.append(W[1:],0)
    delta=y+W*(1-gamma)-(y_next-W_next*gamma)
    return sum(W)-sum(delta)-L

def constraint2(z):
    y = np.cumsum(z)
    D_n = (2*1852*np.tan(alpha)+110)-y[-1]*np.tan(alpha)
    W_n=(np.sin(theta)*(np.cos(alpha)**2))/(np.cos(theta/2+alpha)*np.cos(theta/2-alpha))*D_n
    return L-y[-1]-W_n*(1-gamma)

constraints=[{'type':'eq','fun':constraint1},
             {'type':'eq','fun':constraint2}]

# Initial guess for z: an array of ones.
n0=10 # initial guess for number of variables in 'z'
z0=np.ones(n0)

# Bounds for z: each element must be between 0 and L.
bounds_z=[(0,L)]*n0

result=minimize(objective,z0,method='SLSQP',bounds=bounds_z,constraints=constraints,options={'disp':True})

print('The minimum value of n is:',len(result.x))


Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 10.0
            Iterations: 33
            Function evaluations: 551
            Gradient evaluations: 29
The minimum value of n is: 10


In [19]:
import cvxpy as cp
import numpy as np
from math import *
# Define constants
L = 1825 * 4
gamma = sin(np.radians(61.5)) / (sin(np.radians(61.5)) + sin(np.radians(58.5)))
alpha = np.radians(1.5)
theta = np.radians(120)

# Number of variables (initial guess)
n0=10 

# Define variables
z = cp.Variable(n0)

# Define y in terms of z (cumulative sum)
y = cp.cumsum(z)

# Define D, W, and delta in terms of y
D = (2 * 1852*tan(alpha) + 110) - y * tan(alpha)
W = (sin(theta) * (cos(alpha) ** 2)) / (cos(theta / 2 + alpha) * cos(theta / 2 - alpha)) * D

delta=y+W*(1-gamma)-(cp.hstack([y[1:],0])-cp.hstack([W[1:],0])*gamma)

constraints=[cp.sum(W)-cp.sum(delta)==L,
             L-y[-1]-(sin(theta)*(cos(alpha)**2))/(cos(theta/2+alpha)*cos(theta/2-alpha))*((2*1852*tan(alpha)+110)-y[-1]*tan(alpha))*(1-gamma)>=0,
             z>=0,
             y<=L]

objective=cp.Minimize(n0)

prob=cp.Problem(objective,constraints)

result=prob.solve()

print('The minimum value of n is:', len(z.value))


TypeError: object of type 'NoneType' has no len()